# Creating a csv-File with images and labels
---
---

In [58]:
import pandas as pd
import glob
import random
import functools
import os
from PIL import Image
from IPython.display import display, clear_output, Image
from ipywidgets import Button, HTML, HBox, Output

# slight variation of https://github.com/agermanidis/pigeon
def annotate(examples,
             options=None,
             include_skip=True,
             display_fn=display):

    examples = list(examples)
   
    annotations = []
    current_index = -1

    def set_label_text():
        nonlocal count_label
        count_label.value = '{} examples annotated, {} examples left'.format(
            len(annotations), len(examples) - current_index
        )

    def show_next():
        nonlocal current_index
        current_index += 1
        set_label_text()
        if current_index >= len(examples):
            for btn in buttons:
                btn.disabled = True
            print('Annotation done.')
            return
        with out:
            clear_output(wait=True)
            display_fn(examples[current_index])

    def add_annotation(annotation):
        annotations.append((examples[current_index], annotation))
        show_next()

    def skip(btn):
        show_next()

    count_label = HTML()
    set_label_text()
    display(count_label)



    buttons = []
    
    for label in options:
        btn = Button(description=label)
        def on_click(label, btn):
            add_annotation(label)
        btn.on_click(functools.partial(on_click, label))
        buttons.append(btn)


    if include_skip:
        btn = Button(description='skip')
        btn.on_click(skip)
        buttons.append(btn)

    box = HBox(buttons)
    display(box)

    out = Output()
    display(out)

    show_next()

    return annotations

In [59]:
image_names=glob.glob("Images/*.jpg")

annotations = annotate(
  image_names, 
  options=['mountain', 'cabin', 'forest', 'beach', 'bergsee', 'desert', 'building'], include_skip=True,
  display_fn=lambda filename: display(Image(filename))
)

HTML(value='0 examples annotated, 735 examples left')

Output()

In [3]:
annotations

[('Images\\023.JPG', 'desert'),
 ('Images\\032.JPG', 'beach'),
 ('Images\\036.JPG', 'beach'),
 ('Images\\10991612_870902366284131_4698480174384260508_o.jpg', 'beach'),
 ('Images\\11080717_898573953516972_7426226399171102221_o.jpg', 'building'),
 ('Images\\134.JPG', 'beach'),
 ('Images\\151.JPG', 'beach'),
 ('Images\\167.JPG', 'desert'),
 ('Images\\168.JPG', 'beach'),
 ('Images\\191662_185181388189569_5408132_o.jpg', 'beach'),
 ('Images\\1973808_898573550183679_2250749955249535106_o.jpg', 'building')]

In [4]:
import csv
#open("annotations.csv", "x")
with open("annotations.csv", "w") as the_file:
    csv.register_dialect("custom", delimiter=",", skipinitialspace=True)
    writer = csv.writer(the_file, dialect="custom")
    writer.writerow(("Image_name", "Label"))
    for tup in annotations:
        writer.writerow(tup)
        #writer.write(tup)

In [5]:
pd.read_csv('annotations.csv')

,Image_name,Label
0,Images\023.JPG,desert
1,Images\032.JPG,beach
2,Images\036.JPG,beach
3,Images\10991612_870902366284131_46984801743842...,beach
4,Images\11080717_898573953516972_74262263991711...,building
5,Images\134.JPG,beach
6,Images\151.JPG,beach
7,Images\167.JPG,desert
8,Images\168.JPG,beach
9,Images\191662_185181388189569_5408132_o.jpg,beach


---
# Palmtree Classifier
---

In [51]:
def resize_and_crop(img_path, size, crop_type='middle'):
    """
    Resize and crop an image to fit the specified size.
    args:
        img_path: path for the image to resize..
        size: `(width, height)` tuple.
        crop_type: can be 'top', 'middle' or 'bottom', depending on this
            value, the image will cropped getting the 'top/left', 'middle' or
            'bottom/rigth' of the image to fit the size.
    raises:
        Exception: if can not open the file in img_path of there is problems
            to save the image.
        ValueError: if an invalid `crop_type` is provided.
    """
    # If height is higher we resize vertically, if not we resize horizontally
    img = Image.open(img_path)
    # Get current and desired ratio for the images
    img_ratio = img.size[0] / float(img.size[1])
    ratio = size[0] / float(size[1])
    #The image is scaled/cropped vertically or horizontally depending on the ratio
    if ratio > img_ratio:
        img = img.resize((size[0], int(size[0] * img.size[1] / img.size[0])),
                Image.ANTIALIAS)
        # Crop in the top, middle or bottom
        if crop_type == 'top':
            box = (0, 0, img.size[0], size[1])
        elif crop_type == 'middle':
            box = (0, (img.size[1] - size[1]) / 2, img.size[0], (img.size[1] + size[1]) / 2)
        elif crop_type == 'bottom':
            box = (0, img.size[1] - size[1], img.size[0], img.size[1])
        else :
            raise ValueError('ERROR: invalid value for crop_type')
        img = img.crop(box)
    elif ratio < img_ratio:
        img = img.resize((size[1] *int( img.size[0] / img.size[1]), size[1]),
                Image.ANTIALIAS)
        # Crop in the top, middle or bottom
        if crop_type == 'top':
            box = (0, 0, size[0], img.size[1])
        elif crop_type == 'middle':
            box = ((img.size[0] - size[0]) / 2, 0, (img.size[0] + size[0]) / 2, img.size[1])
        elif crop_type == 'bottom':
            box = (img.size[0] - size[0], 0, img.size[0], img.size[1])
        else :
            raise ValueError('ERROR: invalid value for crop_type')
        img = img.crop(box)
    else :
        img = img.resize((size[0], size[1]),
                Image.ANTIALIAS)
        # If the scale is the same, we do not need to crop
    img.save(img_path)

In [60]:
image_names=glob.glob("Bilder/*.jpg")
#for image in image_names:
#   resize_and_crop(image, (100,100))

In [61]:
palmtree_annotations = annotate(
  image_names, 
  options=['nope', 'palmtree'], include_skip=True,
  display_fn=lambda filename: display(Image(filename))
)

HTML(value='0 examples annotated, 714 examples left')

Output()

In [64]:
palmtree_annotations

[('Bilder\\00000PORTRAIT_00000_BURST20200112193145840.jpg', 'nope'),
 ('Bilder\\00000PORTRAIT_00000_BURST20200123092647375.jpg', 'nope'),
 ('Bilder\\00000PORTRAIT_00000_BURST20200523193045802.jpg', 'nope'),
 ('Bilder\\023.JPG', 'nope'),
 ('Bilder\\028.jpg', 'nope'),
 ('Bilder\\032.JPG', 'palmtree'),
 ('Bilder\\036.JPG', 'palmtree'),
 ('Bilder\\10295222_873765872664447_6139990852376371238_o.jpg', 'nope'),
 ('Bilder\\10608430_843944268979941_2106562515853277705_o.jpg', 'nope'),
 ('Bilder\\10808073_10152947091749758_938191433_n.jpg', 'nope'),
 ('Bilder\\10991612_870902366284131_4698480174384260508_o.jpg', 'nope'),
 ('Bilder\\11080717_898573953516972_7426226399171102221_o.jpg', 'nope'),
 ('Bilder\\130.JPG', 'nope'),
 ('Bilder\\134.JPG', 'palmtree'),
 ('Bilder\\151.JPG', 'palmtree'),
 ('Bilder\\167.JPG', 'nope'),
 ('Bilder\\168.JPG', 'palmtree'),
 ('Bilder\\191662_185181388189569_5408132_o.jpg', 'nope'),
 ('Bilder\\1973808_898573550183679_2250749955249535106_o.jpg', 'nope'),
 ('Bilder\\1992

In [70]:
palmtree_annotations[576][1]='palmtree'

TypeError: 'tuple' object does not support item assignment

In [84]:
import csv
#open("annotations.csv", "x")
with open("palmtree_annotations.csv", "w") as the_file:
    csv.register_dialect("custom", delimiter=",", skipinitialspace=True)
    writer = csv.writer(the_file, dialect="custom")
    writer.writerow(("Image_name", "Label"))
    for tup in palmtree_annotations:
        writer.writerow(tup)
        #writer.write(tup)

In [73]:
palm=pd.read_csv('palmtree_annotations.csv')

In [82]:
palm["Label"].iloc[576]='palmtree'

In [83]:
palm

,Image_name,Label
0,Bilder\00000PORTRAIT_00000_BURST20200112193145...,nope
1,Bilder\00000PORTRAIT_00000_BURST20200123092647...,nope
2,Bilder\00000PORTRAIT_00000_BURST20200523193045...,nope
3,Bilder\023.JPG,nope
4,Bilder\028.jpg,nope
...,...,...
649,Bilder\IMG_20210325_190254.jpg,nope
650,Bilder\IMG_20210402_101847.jpg,nope
651,Bilder\IMG_2745.JPG,nope
652,Bilder\IMG_2776.JPG,palmtree
